In [3]:
import os
import zipfile as zf

# Path to the repaired ZIP file
zip_file_path = "PartB_DFU_dataset - Copy.zip"
extract_path = "DFU_dataset"

if os.path.exists(zip_file_path):
    try:
        with zf.ZipFile(zip_file_path, 'r') as files:
            files.extractall(extract_path)
        print(f"Extraction completed successfully to '{extract_path}'")
    except zf.BadZipFile:
        print("Error: The ZIP file is corrupted.")
    except OSError as e:
        print(f"OS error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print(f"Error: The file '{zip_file_path}' does not exist.")


Extraction completed successfully to 'DFU_dataset'


In [4]:
import numpy as np # linear algebra
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
# Define the root directory where your image folders are located
root_directory = "/DFU_dataset/PartB_DFU_dataset - Copy"

# Initialize lists to store image paths and corresponding class labels for both datasets
image_paths_ischaemia = []
categories_ischaemia = []
image_paths_infection = []
categories_infection = []

# Iterate over each class and its subdirectories
for class_name in ["Infection", "Ischaemia"]:
    for augmentation_type in ["Aug-Negative", "Aug-Positive"]:
        folder_path = os.path.join(root_directory, class_name, augmentation_type)
        category = f"{class_name.lower()}{'pov' if 'Positive' in augmentation_type else 'neg'}"
        
        # Iterate over image files in the current directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".jpg"):  # Assuming images are jpg format
                image_path = os.path.join(folder_path, file_name)
                if class_name == "Ischaemia":
                    image_paths_ischaemia.append(image_path)
                    categories_ischaemia.append("ischemia" if "Positive" in augmentation_type else "non-ischemia")
                elif class_name == "Infection":
                    image_paths_infection.append(image_path)
                    categories_infection.append("infection" if "Positive" in augmentation_type else "non-infection")

# Create DataFrames for each dataset
df_ischaemia = pd.DataFrame({"category": categories_ischaemia, "image_path": image_paths_ischaemia})
df_infection = pd.DataFrame({"category": categories_infection, "image_path": image_paths_infection})

# Label encoding for Ischaemia dataset
label_encoder_ischaemia = LabelEncoder()
df_ischaemia['Class_Label'] = label_encoder_ischaemia.fit_transform(df_ischaemia['category'])
print("Ischaemia Class Mapping:")
for class_label, numerical_label in zip(df_ischaemia['category'].unique(), df_ischaemia['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Label encoding for Infection dataset
label_encoder_infection = LabelEncoder()
df_infection['Class_Label'] = label_encoder_infection.fit_transform(df_infection['category'])
print("Infection Class Mapping:")
for class_label, numerical_label in zip(df_infection['category'].unique(), df_infection['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Shuffle both DataFrames
df_ischaemia = df_ischaemia.sample(frac=1).reset_index(drop=True)
df_infection = df_infection.sample(frac=1).reset_index(drop=True)

# Helper function to load and process images
def load_images(df):
    images = []
    target_labels = []   
    for index, row in df.iterrows():
        image = Image.open(row['image_path'])
        image_array = np.array(image.resize((224, 224)))  # Resize image to fit MobileNet input size
        images.append(image_array)
        target_labels.append(row['Class_Label'])
    return np.array(images), np.array(target_labels)

# Load images for both datasets
images_ischaemia, target_labels_ischaemia = load_images(df_ischaemia)
images_infection, target_labels_infection = load_images(df_infection)

print("Shape of Ischaemia images array:", images_ischaemia.shape)
print("Shape of Ischaemia target labels array:", target_labels_ischaemia.shape)
print("Shape of Infection images array:", images_infection.shape)
print("Shape of Infection target labels array:", target_labels_infection.shape)

# Split the Ischaemia dataset
X_train_ischaemia, X_test_ischaemia, y_train_ischaemia, y_test_ischaemia = train_test_split(
    images_ischaemia, target_labels_ischaemia, test_size=0.3, random_state=42)
X_val_ischaemia, X_test_ischaemia, y_val_ischaemia, y_test_ischaemia = train_test_split(
    X_test_ischaemia, y_test_ischaemia, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

# Split the Infection dataset
X_train_infection, X_test_infection, y_train_infection, y_test_infection = train_test_split(
    images_infection, target_labels_infection, test_size=0.3, random_state=42)
X_val_infection, X_test_infection, y_val_infection, y_test_infection = train_test_split(
    X_test_infection, y_test_infection, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

print("Ischaemia Training set shape:", X_train_ischaemia.shape, y_train_ischaemia.shape)
print("Ischaemia Validation set shape:", X_val_ischaemia.shape, y_val_ischaemia.shape)
print("Ischaemia Test set shape:", X_test_ischaemia.shape, y_test_ischaemia.shape)
print("Infection Training set shape:", X_train_infection.shape, y_train_infection.shape)
print("Infection Validation set shape:", X_val_infection.shape, y_val_infection.shape)
print("Infection Test set shape:", X_test_infection.shape, y_test_infection.shape)

Ischaemia Class Mapping:
non-ischemia: 1
ischemia: 0
Infection Class Mapping:
non-infection: 1
infection: 0
Shape of Ischaemia images array: (9870, 224, 224, 3)
Shape of Ischaemia target labels array: (9870,)
Shape of Infection images array: (5890, 224, 224, 3)
Shape of Infection target labels array: (5890,)
Ischaemia Training set shape: (6909, 224, 224, 3) (6909,)
Ischaemia Validation set shape: (2220, 224, 224, 3) (2220,)
Ischaemia Test set shape: (741, 224, 224, 3) (741,)
Infection Training set shape: (4123, 224, 224, 3) (4123,)
Infection Validation set shape: (1325, 224, 224, 3) (1325,)
Infection Test set shape: (442, 224, 224, 3) (442,)


In [10]:
############################################################################
# ELEPHANT HERDING OPTIMIZATION (EHO) Algorithm Implementation
############################################################################

def initial_variables(size, min_values, max_values, target_function, start_init=None):
    dim = len(min_values)
    
    if start_init is not None:
        start_init = np.atleast_2d(start_init)
        n_rows = size - start_init.shape[0]
        if n_rows > 0:
            rows = np.random.uniform(min_values, max_values, (n_rows, dim))
            start_init = np.vstack((start_init[:, :dim], rows))
        else:
            start_init = start_init[:size, :dim]
        
        fitness_values = np.array([target_function(ind) for ind in start_init])
        # Ensure fitness_values is a 2D array for concatenation
        if fitness_values.ndim == 1:
            fitness_values = fitness_values.reshape(-1, 1)
        print(f"Shape of start_init: {start_init.shape}")
        print(f"Shape of fitness_values: {fitness_values.shape}")
        population = np.hstack((start_init, fitness_values))
    else:
        population = np.random.uniform(min_values, max_values, (size, dim))
        fitness_values = np.array([target_function(ind) for ind in population])
        # Ensure fitness_values is a 2D array for concatenation
        if fitness_values.ndim == 1:
            fitness_values = fitness_values.reshape(-1, 1)
        print(f"Shape of population: {population.shape}")
        print(f"Shape of fitness_values: {fitness_values.shape}")
        population = np.hstack((population, fitness_values))
    
    return population



def update_herd(population, alpha, beta, best_elephant, idx_b, idx_w, min_values, max_values, target_function):
    old_population = np.copy(population)
    cut = population.shape[0]
    dim = len(min_values)
    for i in range(0, cut):
        if i != idx_b and i != idx_w:
            r = np.random.rand(dim)
            population[i, :-1] = np.clip(old_population[i, :-1] + alpha * (best_elephant[:-1] - old_population[i, :-1]) * r, min_values, max_values)
        elif i == idx_b:
            center = np.mean(old_population[:, :-1], axis=0)
            population[i, :-1] = np.clip(beta * center, min_values, max_values)
        elif i == idx_w:
            random_values = np.random.rand(dim)
            population[i, :-1] = np.clip(min_values + (max_values - min_values) * random_values, min_values, max_values)
    
    # Calculate fitness values
    fitness_values = np.array([target_function(ind) for ind in population[:, :-1]])
    population[:, -1] = fitness_values
    
    idx_b = np.argmin(population[:, -1])
    idx_w = np.argmax(population[:, -1])
    if population[idx_b, -1] < best_elephant[-1]:
        best_elephant = np.copy(population[idx_b, :])
    return population, best_elephant, idx_b, idx_w


# Function: EHO
def elephant_herding_optimization(size=50, alpha=0.5, beta=0.1, min_values=[0.0001], max_values=[0.01], generations=5000, target_function=target_function, verbose=True, start_init=None, target_value=None):
    population = initial_variables(size, min_values, max_values, target_function, start_init)
    idx_b = np.argmin(population[:, -1])
    idx_w = np.argmax(population[:, -1])
    best_elephant = population[idx_b, :-1]
    min_values = np.array(min_values)
    max_values = np.array(max_values)
    count = 0
    while count <= generations:
        if verbose:    
            print('Generation: ', count, ' f(x) = ', best_elephant[-1])
        population, best_elephant, idx_b, idx_w = update_herd(population, alpha, beta, best_elephant, idx_b, idx_w, min_values, max_values, target_function)
        if target_value is not None:
            if best_elephant[-1] <= target_value:
                count = 2 * generations
            else:
                count = count + 1
        else:
            count = count + 1
    return best_elephant

############################################################################

In [7]:
import numpy as np
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dropout, Dense, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow import keras

# Define the CNN model-building function
def build_cnn_model(hp):
    base_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        BatchNormalization(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        BatchNormalization(),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(2, activation='softmax')  # Assuming 3 classes for classification
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp['learning_rate']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Modify initial_variables and target_function for two hyperparameters (learning rate and batch size)

def target_function(hyperparams):
    # Extract the learning rate and batch size from hyperparams
    learning_rate = hyperparams[0]
    batch_size = int(hyperparams[1])
    
    hp = {'learning_rate': float(learning_rate)}
    
    # Build CNN model
    model = build_cnn_model(hp)
    
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
    
    # Train model with the given batch size
    history = model.fit(
        X_train_infection, y_train_infection,
        batch_size=batch_size,  # Use batch size as part of hyperparameters
        epochs=10,
        validation_data=(X_val_infection, y_val_infection),
        callbacks=[early_stopping, reduce_lr],
        verbose=0
    )
    
    # Return the validation loss as a scalar value
    val_loss = min(history.history['val_loss'])
    return val_loss

# Test the updated initial_variables function for tuning learning rate and batch size


def simple_target_function(hyperparams):
    return np.array([hyperparams[0] ** 2])  # Return a 1D array

# Define the new ranges for both learning rate and batch size
min_values = [0.0001, 8]  # min learning rate and batch size
max_values = [0.01, 128]  # max learning rate and batch size

# You can still use the EHO algorithm as before
population = initial_variables(3, min_values, max_values, target_function)
print(population)

2024-09-26 11:40:22.521554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 11:40:22.637306: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 11:40:22.663839: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 11:40:22.871586: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-26 11:40:27.524299: I tensorflow/core/common_

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


I0000 00:00:1727350840.917634    1023 service.cc:146] XLA service 0x7d4adc036e60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727350840.917703    1023 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
I0000 00:00:1727350840.917708    1023 service.cc:154]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
I0000 00:00:1727350840.917715    1023 service.cc:154]   StreamExecutor device (2): NVIDIA GeForce RTX 4090, Compute Capability 8.9
I0000 00:00:1727350840.917723    1023 service.cc:154]   StreamExecutor device (3): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-09-26 11:40:41.282133: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-26 11:40:42.722109: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
I0000

Shape of population: (3, 2)
Shape of fitness_values: (3, 1)
[[1.43834495e-03 1.21196392e+01 4.89673585e-01]
 [2.75789902e-03 1.11764507e+01 5.00965714e-01]
 [8.72817790e-03 1.15984701e+02 4.50693369e-01]]


In [11]:
# Run Elephant Herding Optimization (EHO)
best_params = elephant_herding_optimization(
    size=5,  # Number of elephants
    alpha=0.5, beta=0.1, 
    min_values=min_values, max_values=max_values,
    generations=5,  # Number of generations
    target_function=target_function
)

# Print the best found hyperparameters
print("Best Hyperparameters: ", best_params)

2024-09-26 11:45:15.657344: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11616', 16 bytes spill stores, 16 bytes spill loads

2024-09-26 11:45:15.782268: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11616', 204 bytes spill stores, 204 bytes spill loads

2024-09-26 11:45:15.783428: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 428 bytes spill stores, 428 bytes spill loads

2024-09-26 11:45:15.836884: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 436 bytes spill stores, 436 bytes spill loads

2024-09-26 11:45:30.483484: I external/local_xla/x

Shape of population: (5, 2)
Shape of fitness_values: (5, 1)
Generation:  0  f(x) =  126.68817109798808


2024-09-26 11:50:59.410981: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 8 bytes spill stores, 8 bytes spill loads

2024-09-26 11:50:59.526211: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 32 bytes spill stores, 32 bytes spill loads

2024-09-26 11:50:59.554854: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 32 bytes spill stores, 32 bytes spill loads

2024-09-26 11:50:59.721629: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11616', 148 bytes spill stores, 148 bytes spill loads

2024-09-26 11:51:14.307324: I external/local_xla/xla/str

Generation:  1  f(x) =  0.4516952931880951


2024-09-26 11:57:24.012550: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11616', 16 bytes spill stores, 16 bytes spill loads

2024-09-26 11:57:24.136645: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 436 bytes spill stores, 436 bytes spill loads

2024-09-26 11:57:24.140089: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11616', 204 bytes spill stores, 204 bytes spill loads

2024-09-26 11:57:24.306750: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 428 bytes spill stores, 428 bytes spill loads

2024-09-26 11:57:24.386155: I external/local_xla/x

...

Generation:  4  f(x) =  0.4516952931880951


2024-09-26 12:19:16.414930: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11616', 16 bytes spill stores, 16 bytes spill loads

2024-09-26 12:19:16.540093: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11616', 204 bytes spill stores, 204 bytes spill loads

2024-09-26 12:19:16.592514: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 428 bytes spill stores, 428 bytes spill loads

2024-09-26 12:19:16.632166: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 436 bytes spill stores, 436 bytes spill loads

2024-09-26 12:19:49.770195: I external/local_xla/x

Generation:  5  f(x) =  0.4472132921218872


2024-09-26 12:21:32.488151: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11616', 16 bytes spill stores, 16 bytes spill loads

2024-09-26 12:21:32.673538: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 428 bytes spill stores, 428 bytes spill loads

2024-09-26 12:21:32.683817: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12418', 436 bytes spill stores, 436 bytes spill loads

2024-09-26 12:21:32.692114: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11616', 204 bytes spill stores, 204 bytes spill loads

2024-09-26 12:21:48.317676: I external/local_xla/x

Best Hyperparameters:  [5.89571174e-03 5.74851187e+01 4.47213292e-01]


In [12]:
# Extract the best hyperparameters from the EHO result
best_learning_rate = best_params[0]  # First parameter: learning rate
best_batch_size = int(best_params[1])  # Second parameter: batch size (convert to int)

# Train the CNN model using the best hyperparameters
hp_best = {'learning_rate': best_learning_rate}
model = build_cnn_model(hp_best)

# Set up callbacks for early stopping and learning rate reduction
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

# Train the model with the best hyperparameters
history = model.fit(
    X_train_infection, y_train_infection,
    epochs=20,  # You can adjust the number of epochs as needed
    batch_size=best_batch_size,
    validation_data=(X_val_infection, y_val_infection),
    callbacks=[early_stopping, reduce_lr],
    verbose=1  # Set to 1 to show training progress
)

# Get the metrics from the training process
train_acc = history.history['accuracy'][-1]
train_loss = history.history['loss'][-1]
val_acc = history.history['val_accuracy'][-1]
val_loss = history.history['val_loss'][-1]

# Print the results
print("Training Accuracy: ", train_acc)
print("Training Loss: ", train_loss)
print("Validation Accuracy: ", val_acc)
print("Validation Loss: ", val_loss)


Epoch 1/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 43s 307ms/step - accuracy: 0.6294 - loss: 0.6985 - val_accuracy: 0.7238 - val_loss: 0.5459 - learning_rate: 0.0059
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7015 - loss: 0.5756 - val_accuracy: 0.7494 - val_loss: 0.5364 - learning_rate: 0.0059
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7247 - loss: 0.5457 - val_accuracy: 0.7638 - val_loss: 0.5045 - learning_rate: 0.0059
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7432 - loss: 0.5176 - val_accuracy: 0.7600 - val_loss: 0.5201 - learning_rate: 0.0059
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7685 - loss: 0.5033 - val_accuracy: 0.7743 - val_loss: 0.4942 - learning_rate: 0.0059
Epoch 6/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7482 - loss: 0.5043 - val_accuracy: 0.7751 - val_loss: 0.4792 - learning_rate: 0.0059
Epoch 7/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.7682 - loss: 0.4886 - val_a